In [ ]:
from pony.orm import *
import os
import json

In [ ]:
from utils import get_xxhash

In [ ]:
db.drop_all_tables(with_all_data=True)

In [ ]:
db = Database(provider="mysql",
                 host="172.16.0.73",
                 user="media",
                 passwd="aterinee",
                 db="media")

class Media(db.Entity):
    xxhash = Required(str, unique=True)
    size = Required(float)
    exif = Optional(LongStr, lazy=True)
    _table_options_ = {
        'ENGINE': 'InnoDB',
        'ENCRYPTION': "'N'",
        'AUTO_INCREMENT': 10
    }
db.generate_mapping(create_tables=True)

In [ ]:
import os 
import exiftool
import get_files
import xxhash
import py.path
import_dir = "/pictures"

In [ ]:
IMAGE_EXT = [".cr2", ".dng", ".jpg"]
VIDEO_EXT = [".mp4"]

In [ ]:
et = exiftool.ExifTool()
et.start()

In [ ]:
media_files = get_files.get_files(import_dir, decorate=py.path.local)
for media_file in media_files:
    try:
        EXIF = et.get_metadata(str(media_file))
        m = Media(
            xxhash=str(media_file),
            size=media_file.size(),
            exif=json.dumps(EXIF)
        )
    
        db.commit()
    except CacheIndexError:
        db.flush()

In [ ]:
import redis
import rq
import configparser
import utils

config = configparser.ConfigParser()
config.read('config.ini')

exif_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=2,
)
hash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=3,
)

for media_file in media_files:
    break
    

In [ ]:
for media_file in media_files:
    xxhash = utils.get_xxhash(str(media_file))
    if hash_cache.get(str(media_file)) is None:
        hash_cache.set(str(media_file), xxhash)
    if exif_cache.get(xxhash) is None:
        EXIF = et.get_metadata(str(media_file))
        exif_cache.set(xxhash, json.dumps(EXIF))

In [ ]:
media_files = get_files.get_files(import_dir, decorate=py.path.local)
for media_file in media_files:
    xxhash_ =hash_cache.get(str(media_file))
    if xxhash_ is None:
        print("\n")
        xxhash = utils.get_xxhash(str(media_file))
        hash_cache.set(str(media_file), xxhash)
    else:
        print(".")
        xxhash=xxhash_.decode("UTF-8")

    if Media.exists(xxhash=xxhash):
        continue

    EXIF_ = exif_cache.get(xxhash)
    if EXIF_ is None:
        print("\n")
        EXIF = et.get_metadata(str(media_file))
        
        hash_cache.set(str(media_file), xxhash)
    else:
        print(".")
        EXIF=json.loEXIF_.decode("UTF-8")

    m = Media(
        xxhash=xxhash,
        size=media_file.size(),
        exif=json.dumps(EXIF),
    )

    db.commit()

In [ ]:
media_file

In [ ]:
xxhash

In [ ]:
!pip install cached_property

In [ ]:
!echo cached_property >> requirements.txt

In [ ]:
from cached_property import cached_property

In [ ]:
from utils import get_xxhash

In [ ]:
class MediaClass(object):
    def __init__(self, path):
        self.path=path
        self.local=py.path.local(path)
        
    @cached_property
    def xxhash(self):
        return get_xxhash(str(self.path))
    
    
        
    def __repr__(self):
        return "{}<{}>".format(self.__class__.__name__, self.path)

In [ ]:
MediaClass(media_file)

In [ ]:
MediaClass(media_file).xxhash

In [ ]:
MediaClass(media_file).local.size()

In [ ]:
import random

In [ ]:
media_file = random.choice(list(media_files))